In [204]:
import numpy as np
import pandas as pd

## Data set

In [205]:
x1 = np.array([ 0.        ,  3.33333333,  6.66666667, 10.        ,  0.        ,
       3.33333333,  6.66666667, 10.        ,  0.        ,  3.33333333,
        6.66666667, 10.        ,  0.        ,  3.33333333,  6.66666667,
       10.        ])
x2 = np.array([0.        , 0.        , 0.        , 0.        , 0.33333333,
       0.33333333, 0.33333333, 0.33333333, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 1.        , 1.        , 1.        ,
       1.        ])
y = np.array([ 1.22353179,  3.57574634,  5.89217837,  8.19889324,  1.96307308,
        4.57556858,  7.48364786,  9.2400334 ,  2.9048125 ,  5.74174653,
        8.19832673, 10.69424417,  4.15380028,  7.12275093,  8.8976015 ,
       11.76820779])

## 1
a) Run a linear regression to find A, B1, and B2.  
b) Calculate SE(B1) and SE(B2)  
c) Calculate a prediction, y_hat, at x1=5, x2=.5 along with $SE(yhat) = \sqrt{RSS/(n-k-1)} / \sqrt{n}$

## a) Define function to calculate A, B1 and V2

In [211]:
pd.DataFrame({"X1":x1, "X2":x2, "Y":y})

,X1,X2,Y
0,0.000000,0.000000,1.223532
1,3.333333,0.000000,3.575746
2,6.666667,0.000000,5.892178
3,10.000000,0.000000,8.198893
4,0.000000,0.333333,1.963073
5,3.333333,0.333333,4.575569
6,6.666667,0.333333,7.483648
7,10.000000,0.333333,9.240033
8,0.000000,0.666667,2.904812
9,3.333333,0.666667,5.741747


In [206]:
def regress_calc(X,y):
  # Calculate the least squares solution using the normal equation

    XTX = np.dot(X.T, X)
    XTy = np.dot(X.T, y)
    beta = np.linalg.solve(XTX, XTy)
 
    return {'intercept':beta[0],'coefficients':beta[1:]}


In [207]:
X = np.vstack([np.ones(len(x1)), x1, x2]).T

In [208]:
results=regress_calc(X,y)
print(results)

{'intercept': 1.0324877889494863, 'coefficients': array([0.73818319, 3.25746294])}


## b) Calculate SE(B1) and SE(B2)

Calculate the residuals

In [209]:
coef=np.array([results['intercept'],results['coefficients'][0],results['coefficients'][1]])

In [210]:
e = y - np.dot(X, coef)
MSE = np.mean(e ** 2)
print("Error:", e)
print("MSE:", MSE)

Error: [ 0.191044    0.08264793 -0.06153067 -0.21542642 -0.15523568 -0.0033508
  0.44411785 -0.26010723 -0.29931726  0.07700615  0.07297572  0.10828254
 -0.13615045  0.37218958 -0.31357048  0.09642519]
MSE: 0.04748665378911754


In [185]:
x = np.column_stack((x1, x2))

# Degrees of freedom
n = len(y)
p = 2
dof = n - p - 1

# Residual sum of squares
rss = np.sum(e ** 2)

# Variance-covariance matrix
XT = np.transpose(x)
XTX = np.dot(XT, x)
vcm = np.linalg.inv(XTX) * rss / dof

# Standard errors of coefficients
se = np.sqrt(np.diagonal(vcm))

print('We have the next SE values for our coefficients:')
print(f'SE(B1):{se[0]}')
print(f'SE(B2):{se[1]}')

We have the next SE values for our coefficients:
SE(B1):0.012652619721547822
SE(B2):0.12652619713413996


## c) Calculate a prediction, y_hat, at x1=5, x2=.5 along with $SE(yhat) = \sqrt{RSS/(n-k-1)} / \sqrt{n}$

## Prediction for x1=5, x2=.5 

In [186]:
yhat=results['intercept']+5*results['coefficients'][0]+0.5*results['coefficients'][1]
print(f'y_hat, at x1=5, x2=.5 is {yhat}')

y_hat, at x1=5, x2=.5 is 6.352135193124998


Calculate $SE(yhat) = \sqrt{RSS/(n-k-1)} / \sqrt{n}$

In [187]:
se_yhat=np.sqrt(rss/dof)/np.sqrt(n)

In [188]:
print(f'SE (yhat) is {se_yhat}')

SE (yhat) is 0.06043855989548238


## 2
a) Calculate m=100 bootstrap sample sets from the original set of 16 samples and, for each bootstrap sample set, estimate A_bs, B1_bs, and B2_bs by regression. (You'll have 100 estimates of each of A_bs, B1_bs, and B2_bs.)  
b) Compute SE(B1_bs) and SE(B2_bs) directly from the m=100 samples. Do they match SE(B1) and SE(B2) from part 1, above?  
c) For each model, A_bs, B1_bs, and B2_bs, calculate a prediction, y_hat_bs, at x1=5, x2=.5. From that set of predictions, compute SE(y_hat_bs). Does it match SE(y_hat) from part 1, above?  
  
  
Hint: Be sure the values of y, x1, and x2 stay aligned when you draw a sample from the original data. Ex: Randomly drawing the i'th sample would yield (y[i],x1[i],x2[i]) -- all with the same i.

a) Calculate m=100 bootstrap sample sets from the original set of 16 samples and, for each bootstrap sample set, estimate A_bs, B1_bs, and B2_bs by regression. (You'll have 100 estimates of each of A_bs, B1_bs, and B2_bs.)

In [189]:
# number of bootstrap samples
m = 100

# arrays to store estimates
A_bs = []
B1_bs = []
B2_bs = []

# bootstrap sampling and regression
for i in range(m):
    # randomly select 16 samples (with replacement) from the original set
    idx = np.random.choice(16, 16, replace=True)
    x1_bs = x1[idx]
    x2_bs = x2[idx]
    y_bs = y[idx]

    X_bs = np.column_stack((np.ones(16), x1_bs, x2_bs))
    result=regress_calc(X_bs,y_bs)
    
    # store estimates
    A_bs.append(result['intercept'])
    B1_bs.append(result['coefficients'][0])
    B2_bs.append(result['coefficients'][1])

In [200]:
df=pd.DataFrame({'intercept':A_bs,'b1':B1_bs,'b2':B2_bs})
df

,intercept,b1,b2
0,1.004626,0.741607,3.184754
1,1.136336,0.722455,3.091356
2,0.988610,0.742711,3.404606
3,1.058033,0.760840,2.926590
4,1.051382,0.741234,3.161095
...,...,...,...
95,1.252382,0.761371,2.793711
96,1.049265,0.748528,3.170919
97,1.136786,0.726776,3.472908
98,1.204325,0.726023,3.170019


b) Compute SE(B1_bs) and SE(B2_bs) directly from the m=100 samples. Do they match SE(B1) and SE(B2) from part 1, above?

In [192]:
SE_B1_bs = df['b1'].std()
SE_B2_bs = df['b2'].std()


print("SE(B1_bs):", SE_B1_bs)
print("SE(B2_bs):", SE_B2_bs)

SE(B1_bs): 0.015976949067804242
SE(B2_bs): 0.1695298715540743


SE(B1_bs) and SE(B2_bs) computed from the bootstrap samples do not match the SE(B1) and SE(B2) computed from the original sample exactly, they are very close and provide a good estimate of the variability in the estimates of the regression coefficients.

c) For each model, A_bs, B1_bs, and B2_bs, calculate a prediction, y_hat_bs, at x1=5, x2=.5. From that set of predictions, compute SE(y_hat_bs). Does it match SE(y_hat) from part 1, above?

In [212]:
new_x1=5
new_x2=0.5

y_hat_bs = []

for i in range(m):
    y_hat_bs.append(df['intercept'][i]+df['b1'][i]*new_x1+df['b1'][i]*new_x2)


In [213]:
df['y_hat(Preds)']=y_hat_bs

In [214]:
df

,intercept,b1,b2,y_hat(Preds)
0,1.004626,0.741607,3.184754,5.083462
1,1.136336,0.722455,3.091356,5.109836
2,0.988610,0.742711,3.404606,5.073520
3,1.058033,0.760840,2.926590,5.242652
4,1.051382,0.741234,3.161095,5.128172
...,...,...,...,...
95,1.252382,0.761371,2.793711,5.439923
96,1.049265,0.748528,3.170919,5.166170
97,1.136786,0.726776,3.472908,5.134052
98,1.204325,0.726023,3.170019,5.197453


In [215]:
se_y_hat_bs = np.std(y_hat_bs)
print('SE(y_hat_bs):', se_y_hat_bs)

SE(y_hat_bs): 0.10328958024482228


It can be observed that the value is slightly larger before the bootstrap which is expected due to the variability induced by the bootstrap.